### preprocessing

In [ ]:
%cd /home/binglun/totopos_testing/data 

In [2]:
%cd /home/binglun/totopos_testing/data 
%load_ext autoreload
%autoreload 2 
import numpy as np
import anndata as ad
import sc
from plotly import express as px
from ripser import ripser
from persim import plot_diagrams

In [ ]:
adata = ad.read_h5ad('E8.5b.h5ad')
adata

In [ ]:
import mygene

mg = mygene.MyGeneInfo()
ensembl_ids = adata.var['features'].tolist()
result = mg.querymany(ensembl_ids, scopes='ensembl.gene', fields='symbol', species='mouse')
id_to_symbol = {entry['query']: entry.get('symbol', None) for entry in result}
adata.var['gene_name'] = adata.var['features'].map(id_to_symbol)

In [9]:
adata.raw.var.rename(columns={'_index': 'index_backup'}, inplace=True)

In [10]:
adata.write_h5ad('E8.5b.h5ad', compression='gzip')

In [ ]:
mt_ribo_filt = True
if mt_ribo_filt:
    #try human 
    if adata.var["gene_name"].str.contains("MT-").any():
        print("Detected human gene name format for mitochondrial genes.")
        a = sc.get_count_stats(adata, mt_prefix="MT-", 
                               ribo_prefix = ("RPS", "RPL"))
    #try mouse 
    elif adata.var["gene_name"].str.contains("mt-").any(): 
        print("Detected mouse gene name format for mitochondrial genes.")
        a = sc.get_count_stats(adata, mt_prefix="mt-", 
                               ribo_prefix = ("Rps", "Rpl"))
    else: #TO-DO allow for arbitrary prefixes
        print("Could not calculate mitochondrial and \
              ribosomal gene content.")
        mt_ribo_filt = False
else:
    a = sc.get_count_stats(adata)

In [12]:
adata.obs['n_counts'] = np.asarray(adata.X.sum(axis = 1))

In [14]:
adata.obs['log_counts'] = np.log10(adata.obs.n_counts)

In [16]:
adata.obs['n_genes'] = np.asarray((adata.X > 0).sum(axis = 1))

In [18]:
mito_genes = adata.var['gene_name'].str.startswith('mt-')

In [30]:
mito_inds = np.where(adata.var['gene_name'].str.startswith('mt-'))[0]

In [31]:
adata.obs["frac_mito"] = adata[:, mito_inds].X.toarray().sum(axis =1) / adata.obs.n_counts

In [33]:
ribo_prefix = ("Rps", "Rpl")
ribo_genes = np.zeros(adata.n_vars, dtype = bool)
for prefix in ribo_prefix:
    ribo_genes_tmp = adata.var['gene_name'].str.startswith(prefix)
    ribo_genes +=ribo_genes_tmp

In [35]:
ribo_inds = np.where(ribo_genes)[0]

In [36]:
adata.obs["frac_ribo"] = adata[:, ribo_inds].X.toarray().sum(axis =1) / adata.obs.n_counts

In [38]:
adata.write_h5ad('E8.5b.h5ad', compression='gzip')

In [ ]:
adata.obs['n_counts'].describe()

In [42]:
a = adata

In [ ]:
import scipy.sparse
if scipy.sparse.issparse(a.X):
    # Using getnnz counts the nonzero entries along the axis (axis=0 for genes)
    gene_cell_counts = a.X.getnnz(axis=0)
else:
    gene_cell_counts = np.sum(a.X > 0, axis=0)

min_cells = 10                         # Gene must be expressed in at least 10 cells
max_cells = 0.9 * a.n_obs            # Gene expressed in >60% of cells is excluded
genes_to_keep = (gene_cell_counts >= min_cells) & (gene_cell_counts <= max_cells)
a = a[:, genes_to_keep]
a

In [ ]:
import scanpy
scanpy.pl.violin(a, ['n_genes', 'n_counts', 'frac_mito', 'frac_ribo'],
    jitter=0.4, multi_panel=True)

In [ ]:
max_frac_mito = 0.2
max_frac_ribo = 0.2

filtered_cells = a[(a.obs['frac_mito'] > max_frac_mito) 
                   | (a.obs['frac_ribo'] > max_frac_ribo)]
print(f"Number of cells with max_frac_mito > {max_frac_mito} \
      or max_frac_ribo > {max_frac_ribo}: {filtered_cells.n_obs} \n")

In [ ]:
a_ = a
mito_ribo_filt = True
if mito_ribo_filt: 
    try: 
        print(f"Mean frac mitochondrial counts \
              {a_.obs.frac_mito.mean():.2f}, \
                ribosomal gene counts {a_.obs.frac_ribo.mean():.2f}")        
        a_ = a_[(a_.obs.frac_mito < max_frac_mito) & \
                (a_.obs.frac_ribo < max_frac_ribo)].copy()
        print(f"Filtered out cells with max.frac of mitochondrial \
              {max_frac_mito}  and {max_frac_ribo} of ribosomal counts.")
    except: 
        print("Could not perform filtering by ribo/mito content.")

print(f"Number of cells after filtering: {a_.n_obs}")

In [55]:
try:
    ada = sc.lognorm_cells(a_)
except: 
    a_.X = a_.X.astype(np.float32)
    ada = sc.lognorm_cells(a_)

In [ ]:
adat = sc.cv_filter(ada, return_highly_variable = True)
print(f"Number of genes after filtering: {adat.n_vars}")

In [62]:
adat.write_h5ad('E8.5b_hvg.h5ad', compression='gzip')

In [ ]:
adat

### totopos

In [ ]:
%load_ext autoreload
%autoreload 2 
import totopos.cells as tpc

n_pcs = tpc.find_pca_cutoff(adat, thres=0.01)
print(f'Number of PCs: {n_pcs}')

In [ ]:
adat

In [ ]:
outlier_mask, scores = tpc.detect_outliers(
    adat, n_pcs=n_pcs, contamination=0.01, max_samples=100000, 
    n_jobs=8, n_estimators=200)

adat.obs['if_outlier_mask'] = outlier_mask
adat.obs['if_outlier_score'] = scores

In [ ]:
tpc.plot_outliers(adat, outlier_mask, scores)

In [ ]:
fig=px.scatter_3d(
    adat.obs,
    # x="UMAP_1", y = "UMAP_2", z = "UMAP_3", 
    x="pc1", y="pc2", z="pc3", 
    color = 'if_outlier_mask', 
    # hover_data=["cell_type", 'E_day', 'author_cell_type'], 
    width=1000, height=800
)
fig.update_traces(marker=dict(size=1))

In [68]:
adat.write_h5ad('E8.5b_hvg_outliers.h5ad', compression='gzip')

### tpc

In [1]:
%cd /home/binglun/totopos_testing/data 
%load_ext autoreload
%autoreload 2 
import numpy as np
import anndata as ad
import pandas as pd
import sc
from plotly import express as px
from ripser import ripser
from persim import plot_diagrams

/home/binglun/totopos_testing/data


In [2]:
adata = ad.read_h5ad('E8.5b_hvg_outliers.h5ad')
adata

AnnData object with n_obs × n_vars = 153073 × 6588
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'sample', 'day', 'group', 'cell_state', 'cell_type', 'somite_stage', 'n_counts', 'log_counts', 'n_genes', 'frac_mito', 'frac_ribo', 'doublet_score', 'predicted_doublet', 'pc1', 'pc2', 'pc3', 'pc4', 'if_outlier_mask', 'if_outlier_score'
    var: 'features', 'gene_name', 'mean', 'log_mean', 'var', 'cv', 'log_cv', 'highly_variable'
    obsm: 'pcs'

In [3]:
ph = ripser(adata.obsm["pcs"][:, :35], n_perm=1000, do_cocycles=True)

In [4]:
import totopos.cells as tpc
topological_loops = tpc.critical_edge_method(adata.obsm["pcs"][:, :35], ph, verbose=True, n_loops=3)

Starting VR graph construction.
Finished VR graph. Starting loop discovery...


  0%|          | 0/3 [00:00<?, ?it/s]

Starting 1-th loop discovery...
Finished computing loop 1 from VR graph.


 33%|███▎      | 1/3 [00:00<00:01,  1.41it/s]

Starting 2-th loop discovery...
Finished computing loop 2 from VR graph.


 67%|██████▋   | 2/3 [00:01<00:00,  1.47it/s]

Starting 3-th loop discovery...
Finished computing loop 3 from VR graph.


100%|██████████| 3/3 [00:02<00:00,  1.42it/s]

Finished critical edge algorithm.


#### plot loops/topocells

In [ ]:
import totopos.viz.cloud as tpv
tpv.quick_loop_summary(topological_loops)

In [ ]:
fig = tpv.plot_loops(
    adata=adata,
    topological_loops=topological_loops,
    title="Topological Loops in Single-Cell Data",
    pcs_viz=(1, 2, 3),  # Use PC1, PC2, PC3
    # color_col='cell_type',  # Color by cell type
    hover_cols=['cell_type', 'sample', 'day'],  # Show in hover
    use_pca=True  # Use PCA coordinates from adata.obsm['pcs']
)

fig.show()

In [ ]:
tpv.summarize_topocells(adata, topological_loops)

In [ ]:
fig1 = tpv.plot_topocells_highlighted(
    adata=adata,
    topological_loops=topological_loops,
    title="All Topocells Highlighted",
    # color_col='cell_type',  # Color topocells by cell type
    hover_cols=['cell_type', 'sample', 'day'],
    pcs_viz=(1, 2, 3),
    dot_size_gray=1,  # Small gray dots
    dot_size_topo=3,  # Larger colored dots
    show_loops=True   # Also show connections
)

fig1.show()

In [ ]:
fig2 = tpv.plot_single_loop_topocells(
    adata=adata,
    topological_loops=topological_loops,
    loop_index=1,  # First loop
    # color_col='somite_stage',
    pcs_viz=(1, 2, 3),
    hover_cols=['cell_type', 'somite_stage', 'day'],
    show_loops=True
)

fig2.show()

#### topogenes

In [5]:
import totopos.genes as tpg

coords = tpg.get_circular_coords(
    topological_loops,
    loop_index=0,
    n_dim=None,        # use all features of that loop
    plot=False,         # if you want to see the persistence diagram
    n_landmarks=1000,
    prime=47,
    cocycle_idx=0,
    standard_range=False
)

/home/binglun/miniconda3/envs/totopos/lib/python3.13/site-packages/dreimac/emcoords.py:179: UserWarning: There are 17427 point not covered by a landmark
  warnings.warn("There are {} point not covered by a landmark".format(nzero))


In [6]:
scores, feat_order = tpg.topo_scores_rep_sampling(
    adata,
    topological_loops,
    coords,
    loop_index=0,
    n_reps=100,
    frac_rep=0.2,
    seed=42
)
# `scores[i]` is the importance of feature i, and `feat_order` lists feature indices
# from most→least important.

In [7]:
# Suppose `scores, sorted_idx = compute_topo_gradient_scores(...)`
top20 = tpg.get_topogenes(adata, feat_order, n_genes=20, gene_name_col='gene_name')
print("Top 20 genes:", top20)

Top 20 genes: ['Robo2', 'Unc5c', 'Kif26b', 'Kcnh7', 'Pcdh7', 'Epha5', 'Smoc1', 'Cdh11', 'Slit2', 'Sox6', 'Tshz2', 'Cacna2d1', 'Ebf1', 'Nrp1', 'Nkain3', 'Kcnq5', 'H19', 'Foxp2', 'Sdk1', 'Frem1']
